# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "/users/brant/Documents/DABC/pandas-resources/schools_complete.csv"
student_data_to_load = "/users/brant/Documents/DABC/pandas-resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
pycityschools = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

#change data types for numeric valuse that will be analyzed. Div does not work with str or int values.
pycityschools['reading_score'] = pycityschools['reading_score'].astype('float').map('{:.0f}'.format)
pycityschools['math_score'] = pycityschools['math_score'].astype('float').map('{:.0f}'.format)
pycityschools['size'] = pycityschools['size'].astype('float').map('{:,.0f}'.format)
pycityschools['budget'] = pycityschools['budget'].astype('float').map('${:,.2f}'.format)
# 
#pycityschools.head()
pycityschools.tail()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,"1,635","$1,043,130.00"
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,"1,635","$1,043,130.00"
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,"1,635","$1,043,130.00"
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,"1,635","$1,043,130.00"
39169,39169,Carolyn Jackson,F,11th,Thomas High School,95,75,14,Charter,"1,635","$1,043,130.00"


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [4]:

schoolcount = pycityschools['school_name'].nunique()
studentcount = pycityschools['Student ID'].nunique()

totalbudget = '${:,.2f}'.format(school_data['budget'].sum())

totalavgmath = '{:,.2f}'.format(student_data['math_score'].mean())
totalavgreading = '{:,.2f}'.format(student_data['reading_score'].mean())

overallpassrate = '{:,.2f}'.format((float(totalavgmath) + float(totalavgreading))/2)

passmathcount = student_data.loc[student_data['math_score'] >= 70]['math_score'].count()
percentpassmath = '{:,.1f}%'.format(passmathcount / studentcount * 100)

passreadcount = student_data.loc[student_data['reading_score'] >= 70]['reading_score'].count()
percentpassread = '{:,.1f}%'.format(passreadcount / studentcount * 100)

schoolscorestats = pd.DataFrame({"Number of Schools":[schoolcount], "Number of Students":[studentcount],
                              "Total Budget":[totalbudget], "Avg Math Score":[totalavgmath],
                                "Avg Reading Score":[totalavgreading],"Overall Passing Rate":[overallpassrate],
                                 "% Passing Math Score":[percentpassmath],"% Passing Reading Score":[percentpassread]
                               },index=['Stats'])

schoolscorestats['Number of Students'] = schoolscorestats['Number of Students'].map('{:,.0f}'.format)

schoolscorestats=schoolscorestats.transpose()

# print(percentpassmath)
# print(percentpassread)
# print(totalavgreading)
# print(overallpassrate)
schoolscorestats

,Stats
Number of Schools,15
Number of Students,"39,170"
Total Budget,"$24,649,428.00"
Avg Math Score,78.99
Avg Reading Score,81.88
Overall Passing Rate,80.44
% Passing Math Score,75.0%
% Passing Reading Score,85.8%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [28]:
# gentotalpurch = purchase_data_file.groupby('Gender')['Price'].sum().reset_index(name= 'Total Purchase')
pycityschoolstats = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
schoolstats = school_data
pycityschoolstats['math_score'] = pycityschoolstats['math_score'].astype('float')
pycityschoolstats['reading_score'] = pycityschoolstats['reading_score'].astype('float')
pycityschoolstats['budget'] = pycityschoolstats['budget'].astype('float')

schoolstats['Per Student Budget'] = schoolstats['budget'] / schoolstats['size']
# df['DataFrame Column'] = pd.to_numeric(df['DataFrame Column'],errors='coerce')

schoolstats['Avg Math Score'] = pycityschoolstats.groupby('school_name')['math_score'].mean()


# schoolstats['Avg Math Score'] = pycityschoolstats['math_score'].transform('mean')

# formatstyle = {'size': '{:,.0f}','budget': '${:,.2f}','Per Student Budget': '${:,.2f}'}

#schoolstats.sort_values("school_name", ascending=True).style.format(formatstyle).hide_index()

schoolstats
# pycityschoolstats.dtypes

,School ID,school_name,type,size,budget,Per Student Budget,Average Math Score,Avg Math Score
0,0,Huang High School,District,2917,1910635,655.0,NaN,NaN
1,1,Figueroa High School,District,2949,1884411,639.0,NaN,NaN
2,2,Shelton High School,Charter,1761,1056600,600.0,NaN,NaN
3,3,Hernandez High School,District,4635,3022020,652.0,NaN,NaN
4,4,Griffin High School,Charter,1468,917500,625.0,NaN,NaN
5,5,Wilson High School,Charter,2283,1319574,578.0,NaN,NaN
6,6,Cabrera High School,Charter,1858,1081356,582.0,NaN,NaN
7,7,Bailey High School,District,4976,3124928,628.0,NaN,NaN
8,8,Holden High School,Charter,427,248087,581.0,NaN,NaN
9,9,Pena High School,Charter,962,585858,609.0,NaN,NaN


* Sort and display the top five schools in overall passing rate

## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [ ]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

## Scores by School Type

* Perform the same operations as above, based on school type.